# PyTorch своими руками

In [1]:
# только numpy, только хардкор
import numpy as np

**Module** — это абстрактный класс, в который мы будем оборачивать слои нашей нейронной сети. Он определяет фундаментальные методы, которые нужны для backpropagation.

Можно думать о нём, как о чёрной коробке, которая принимает `input` (какой-то тензор) и возвращает `output` (тоже какой-то тензор), а также у.

У него есть два основных метода — `forward` и `backward`. Первый просто считает.

Каждый модуль — это своя независимая коробка.

В духе лучших принципов ООП, мы хотим абстрагироваться от .

`forward` сохраняет то, что он насчитал — свой `output`.

Нам будет проще, если мы будем сохранять `output` — нам это понадобится при обратном проходе.

**Понять код Module крайне важно**.

Параметр — это что-то, что можно обучать. Он должен быть доступен оптимизатору, а оптимизатор не должен знать, как всё у нас внутри работает. Поэтому градиенты считать тоже нужно нам.

### Forward

Прямой прогон прост. Операция зависит от слоя.

### Backward

Обратный прогон долен делать две вещи:

1. Посчитать и сложить

### train / eval

Некоторые слои ведут себя по-разному при обучении и реальных прогонах.

Например, `BatchNorm` и `DropOut`.

In [2]:
class Module():
    def __init__ (self):
        self._train = True
    
    def forward(self, input):
        raise NotImplementedError

    def backward(self,input, gradOutput):
        raise NotImplementedError
    
    def parameters(self):
        'Возвращает список собственных параметров.'
        return []
    
    def get_grad_parameters(self):
        'Возвращает список градиентов для своих параметров.'
        return []
    
    def train(self):
        self._train = True
    
    def eval(self):
        self._train = False

Это **абстрактный класс** — от него наследуются другие, в которых есть настоящая реализация.

# Sequential

**Sequential** будет оборачивать список модулей и выполнять их последовательно.

Это своего рода контейнер, внутри которого есть какой-то пайплайн.

Можно даже засовывать один Sequential внутри другого.

Многие не знают, но в питоне почти всегда для итерирования используется не **deep copy**, а **shallow copy**. Это делается для экономии памяти.

In [3]:
class Sequential(Module):
    def __init__ (self, *layers):
        super().__init__()
        self.layers = layers

    def forward(self, input):
        """
        Прогоните данные последовательно по всем слоям:
        
            y[0] = layers[0].forward(input)
            y[1] = layers[1].forward(y_0)
            ...
            output = module[n-1].forward(y[n-2])   
            
        Это должен быть просто небольшой цикл: for layer in layers...
        
        Хранить выводы ещё раз не надо: вспомните -- они сохраняются внутри слоев после forward.
        """

        # ...
        
        return self.output

    def backward(self, input, grad_output):
        """
        Backward -- это как forward, только наоборот. (с)
        
        Предназначение backward — посчитать посчитать градиенты для собственных параметров и передать градиент относительно своего входа.
        
        Внутри параметров модули сами позаботятся о своих параметрах. Нам же нужно позаботиться о передачи градиента.
         
            g[n-1] = layers[n-1].backward(y[n-2], grad_output)
            g[n-2] = layers[n-2].backward(y[n-3], g[n-1])
            ...
            g[1] = layers[1].backward(y[0], g[2])   
            grad_input = layers[0].backward(input, g[1])
        
        Тут цикл будет уже посложнее.
        """
        # ...
        
        return grad_input
      
    def parameters(self):
        'Можно просто сконкатенировать все параметры в один список.'
        return [l.parameters() for l in self.layers]
    
    def grad_parameters(self):
        'Можно просто сконкатенировать все градиенты в один список.'
        return [l.parameters() for l in self.layers]

# Слои

Приступим к реализации содержательной части — самих слоев.

Работать мы будем с векторами.

Все операции обучно формулируются так, что они не зависят друг от друга (кроме `BatchNorm`, но об этом позже).

Во всех фреймворках . Мы тоже будем привыкать к этому.

Начнем с основного: линейный слой. Он же афинный, он же fully-conected.

На вход всех слоев будет подаваться матрица размера `batch_size` $\times$ `n_features`.

In [4]:
class Linear(Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
       
        # на самом деле, очень важно, как оно инициализируется
        stdv = 1./np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size=(dim_out, dim_in))
        self.b = np.random.uniform(-stdv, stdv, size=dim_out)
        
        self.grad_W = np.zeros_like(self.W)
        self.grad_b = np.zeros_like(self.b)
        
    def forward(self, input):
        # ...      
        return self.output
    
    def backward(self, input, grad_output):
        # ...
        return self.gradInput
    
    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)
        
    def parameters(self):
        return [self.W, self.b]
    
    def grad_parameters(self):
        return [self.gradW, self.gradb]
    
    def __repr__(self):
        s = self.W.shape
        q = 'Linear %d -> %d' %(s[1],s[0])
        return q

## Функции активации

**ReLU** — одна из самых простых функций активации. 

In [8]:
class ReLU(Module):
    def __init__(self):
         super().__init__()
    
    def updateOutput(self, input):
        self.output = np.maximum(input, 0)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.multiply(gradOutput , input > 0)
        return self.gradInput
    
    def __repr__(self):
        return "ReLU"

Implement [**Leaky Rectified Linear Unit**](http://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29%23Leaky_ReLUs). Expriment with slope. 

In [9]:
class LeakyReLU(Module):
    def __init__(self, slope = 0.03):
        super().__init__()
            
        self.slope = slope
        
    def updateOutput(self, input):
        # Your code goes here. ################################################
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        return self.gradInput
    
    def __repr__(self):
        return "LeakyReLU"

Софтмакс — самый сложный с точки зрения написания `backward`. Хотя, как и все, занимает 5 строчек.

$$ SoftMax(x_k) = \frac{e^{x_k}}{\sum_{i=1}^n e^{x_i} }$$

In [ ]:
class SoftMax(Module):
    def __init__(self):
         super().__init__()
    
    def forward(self, input):
        # важная деталь: если входы большие,
        # то экспоненты будут ещё больше
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))
        
        # Your code goes here. ################################################
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        return self.gradInput
    
    def __repr__(self):
        return "SoftMax"

## Регуляризация

Реализуйте [**дропаут**](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). Идея простая: просто помножьте входные данные на случайную маску того же размера. Сгенерировать маску можно через `np.random.binomial`.

Дропаут клёвый, и его обычно хватает как единственного регуляризатора. Если вы заметите, что сеть оверфитится — просто добавьте его побольше.

Заметитьте, что у него разное поведение в `train` и `eval` режимах. При `eval` он не должен делать ничего, а в `train` помимо маски нужно ещё домножить вход на $\frac{1}{1-p}$, чтобы ожидание не изменилось.

In [ ]:
class Dropout(Module):
    def __init__(self, p=0.5):
        super().__init__()
        
        self.p = p
        self.mask = None
        
    def forward(self, input):
        self.output = 
        mask = # ...
        self.output = # ...
        return self.output
    
    def backward(self, input, grad_output):
        # ...
        return self.gradInput

## Критерии

Критерии — это специальные функции, которые меряют качество, имея реальные данные и предсказанные.

По сути это тоже модули. Их следует отделять от модулей, котому что у них нет `train` / `eval`, а `backward` не требует `grad_output` — эта вершина и так конечная в вычислительном графе.

Также нам не понадобится сохранять output.

In [8]:
class Criterion():        
    def forward(self, input, target):
        raise NotImplementedError

    def backward(self, input, target):
        raise NotImplementedError

Реализуем $L_2$-норму: просто средняя квадратичная ошибка.

Обратите внимание, что в критериях мы делим итоговую цифру на размер батча — мы не хотим, чтобы функция потерь зависела от количества примеров.

In [5]:
import numpy as np

In [9]:
class MSE(Criterion):
    def forward(self, input, target):
        batch_size = input.shape[0]
        self.output = np.sum(np.power(input - target, 2)) / batch_size
        return self.output
 
    def backward(self, input, target):
        self.gradInput  = (input - target) * 2 / input.shape[0]
        return self.gradInput

Ваша задача посложнее: вам нужно реализовать кроссэнтропию — стандартная функция потерь для классификации.

Помните, что лэйблы в one-hot.

Напоминаем интуицию за принципом максимального правдоподобия: максимизируем произведение прсказанных вероятностей реальных событий $ L = \prod p_i $.

Произведение оптимизировать очень не удобно, и поэтому воспользуемся следующим трюком: возьмем логарифм (любой, ведь все логарифмы отличаются в константу раз) и будем максимизировать сумму:

$$ \log L = \log \prod p_i = \sum \log p_i $$

Эту штуку называют кроссэнтропией. Такое название пошло из теории информации, но нам пока знать это не надо.

Для удобноства вместо чисел — от 0 до 9 — сконвертируем их в вектора размера 10, где будет стоять единица в нужном месте (такая кодировка называется one-hot).

In [14]:
class Crossentropy(Criterion):
    def __init__(self):
        super().__init__()
        
    def forward(self, input, target): 
        # можно сделать такой трюк, чтобы нигде не было взятий логарифма от нуля:
        eps = 1e-9
        input_clamp = np.clip(input, eps, 1 - eps)
        
        # ...
        return self.output

    def backward(self, input, target):
        # Use this trick to avoid numerical errors
        input_clamp = np.maximum(1e-15, np.minimum(input, 1 - 1e-15) )
                
        # ....
        return self.gradInput